# Exercise 9

There is no dataset in tfds called SketchRNN. There is the "quickdraw_bitmap" dataset (SketchRNN is a network by the Google Magenta team that was trained on this data), which was too big for me to handle on my laptop, so I'll skip this one.

# Exercise 10

Download the mentioned file. I put it in /15-data/jsb/ and unzipped it with the command:

    tar -xvzf jsb_chorales.tgz

In [154]:
# some imports

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow_datasets as tfds

In [155]:
# paths

import os

dataset_path = os.path.join("15-data", "jsb")
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")
test_path = os.path.join(dataset_path, "test")

train_files = [train_path + os.path.sep + f for f in os.listdir(train_path)]
valid_files = [valid_path + os.path.sep + f for f in os.listdir(valid_path)]
test_files = [test_path + os.path.sep + f for f in os.listdir(test_path)]

In [156]:
# import the csv's

train_data = []
valid_data = []
test_data = []

for file in train_files:
    train_data.append(pd.read_csv(file).to_numpy())
for file in valid_files:
    valid_data.append(pd.read_csv(file).to_numpy())
for file in test_files:
    test_data.append(pd.read_csv(file).to_numpy())

In [157]:
max_train_length = max(len(l) for l in train_data)
max_valid_length = max(len(l) for l in valid_data)
max_test_length = max(len(l) for l in test_data)

max_length = max((max_train_length, max_valid_length, max_test_length))

print("maximum length of samples is {}".format(max_length))

maximum length of samples is 640


In [158]:
# we build the train/valid/test sets. because the samples are of non-uniform length we'll use a masking layer
# therefore, we prepend non-maximum-length samples with zeros.

def build_set(data):
    n_samples = len(data)
    set = np.empty((n_samples, max_length, 4))
    set[:, :, :] = -1.0   # padding value
    for idx, sample in enumerate(data):
        set[idx, -1*len(sample):, :] = sample / 128.0   # max MIDI
    return set

X_train = build_set(train_data)
X_valid = build_set(valid_data)
X_test = build_set(test_data)

In [159]:
# we'll be trying to predict one timestep ahead

def build_target(X):
    target = np.zeros(X.shape)
    target[:, :-1, :] = X[:, 1:, :]
    return target

Y_train = build_target(X_train)
Y_valid = build_target(X_valid)
Y_test = build_target(X_test)

In [160]:
# cut the last timestep, we have no prediction there

X_train, Y_train = X_train[:, :-1, :], Y_train[:, :-1, :]
X_valid, Y_valid = X_valid[:, :-1, :], Y_valid[:, :-1, :]
X_test, Y_test = X_test[:, :-1, :], Y_test[:, :-1, :]

In [161]:
# build a model

model = keras.models.Sequential([
    keras.layers.GRU(100, return_sequences=True),
    keras.layers.GRU(100, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(4))
])

In [167]:
def last_time_step_error(Y_true, Y_pred):
    return tf.math.reduce_sum(tf.math.abs(Y_true[:, -1] - Y_pred[:, -1]))

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_error])

In [169]:
history = model.fit(X_train, Y_train, epochs=50, 
                    validation_data=(X_valid, Y_valid))

Train on 229 samples, validate on 76 samples
Epoch 1/50
229/229 [==============================] - 29s 126ms/sample - loss: 0.0037 - last_time_step_error: 1.8852 - val_loss: 0.0039 - val_last_time_step_error: 1.6727
Epoch 2/50
229/229 [==============================] - 31s 133ms/sample - loss: 0.0036 - last_time_step_error: 1.9313 - val_loss: 0.0039 - val_last_time_step_error: 1.6742
Epoch 3/50
229/229 [==============================] - 17s 74ms/sample - loss: 0.0036 - last_time_step_error: 1.8492 - val_loss: 0.0039 - val_last_time_step_error: 1.6715
Epoch 4/50
229/229 [==============================] - 27s 118ms/sample - loss: 0.0036 - last_time_step_error: 1.8840 - val_loss: 0.0038 - val_last_time_step_error: 1.6315
Epoch 5/50
229/229 [==============================] - 35s 151ms/sample - loss: 0.0036 - last_time_step_error: 1.7998 - val_loss: 0.0038 - val_last_time_step_error: 1.6380
Epoch 6/50
229/229 [==============================] - 25s 108ms/sample - loss: 0.0036 - last_time_ste

229/229 [==============================] - 15s 64ms/sample - loss: 0.0035 - last_time_step_error: 0.8590 - val_loss: 0.0037 - val_last_time_step_error: 0.7960
Epoch 49/50
229/229 [==============================] - 27s 119ms/sample - loss: 0.0035 - last_time_step_error: 0.8366 - val_loss: 0.0037 - val_last_time_step_error: 0.8144
Epoch 50/50
229/229 [==============================] - 26s 115ms/sample - loss: 0.0035 - last_time_step_error: 0.8141 - val_loss: 0.0037 - val_last_time_step_error: 0.7550


As a metric, we're looking at a sum of absolute differences between the prediction and target vector. We see that the model is training - this would me more fun on a GPU for sure. The network is training and the metric looks to do the right thing. This is enough for me & I'll call it a day, while there could a million things be tweaked for sure. Let's just try a prediction and look at the correct values:

In [180]:
model.predict(X_test)[0,-22:-2,:] * 128

array([[62.670017, 59.33638 , 56.684753, 43.57272 ],
       [62.81531 , 58.861008, 56.104836, 44.497883],
       [62.896473, 58.65264 , 55.219753, 44.342564],
       [62.676994, 58.227837, 55.19636 , 45.345043],
       [63.316914, 58.71766 , 55.280937, 45.42322 ],
       [62.310215, 58.345566, 54.967216, 44.695   ],
       [62.930794, 58.79892 , 55.094444, 44.06912 ],
       [61.078457, 58.00331 , 54.428917, 41.779564],
       [61.88028 , 58.522095, 54.872017, 41.169178],
       [61.191967, 57.972366, 54.286438, 40.381042],
       [61.66104 , 58.18567 , 54.419334, 40.30004 ],
       [61.904984, 58.56436 , 54.75032 , 40.337906],
       [61.962124, 58.626915, 54.956005, 40.350636],
       [61.154655, 57.328278, 53.560703, 39.72917 ],
       [61.478954, 57.402466, 52.970367, 39.47121 ],
       [61.804436, 58.046852, 53.755157, 40.320908],
       [62.068935, 58.161697, 54.0656  , 40.201065],
       [61.916653, 58.144226, 53.931763, 39.927032],
       [61.902386, 58.206375, 54.06535 , 40.09

In [181]:
Y_test[0,-22:-2,:] * 128

array([[62., 57., 55., 45.],
       [62., 57., 54., 45.],
       [62., 55., 55., 46.],
       [62., 57., 55., 46.],
       [62., 58., 55., 43.],
       [62., 58., 55., 43.],
       [62., 58., 55., 38.],
       [62., 58., 55., 38.],
       [62., 57., 54., 38.],
       [62., 57., 54., 38.],
       [62., 58., 55., 38.],
       [62., 58., 55., 38.],
       [62., 55., 52., 38.],
       [62., 55., 52., 38.],
       [62., 57., 54., 38.],
       [62., 57., 54., 38.],
       [62., 57., 54., 38.],
       [62., 57., 54., 38.],
       [62., 57., 54., 38.],
       [62., 57., 54., 38.]])